In [51]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack

# IMPORTING ALL THE NECESSARY LIBRARIES AND PACKAGES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [2]:
chunksize = 10000
business_df = pd.DataFrame()

# Iterate over the file in chunks and concatenate the results
for chunk in pd.read_json('/Users/Ethan Vaz Falcao/Downloads/Yelp_data/yelp_academic_dataset_business.json', lines=True, chunksize=chunksize):
    business_df = pd.concat([business_df, chunk], ignore_index=True)

review_df = pd.DataFrame()
# Iterate over the file in chunks and concatenate the results
for chunk in pd.read_json('/Users/Ethan Vaz Falcao/Downloads/Yelp_data/yelp_academic_dataset_review.json', lines=True, chunksize=chunksize):
    review_df = pd.concat([review_df, chunk], ignore_index=True)

tip_df = pd.DataFrame()

# Iterate over the file in chunks and concatenate the results
for chunk in pd.read_json('/Users/Ethan Vaz Falcao/Downloads/Yelp_data/yelp_academic_dataset_tip.json', lines=True, chunksize=chunksize):
    tip_df = pd.concat([tip_df, chunk], ignore_index=True)

In [55]:
import pandas as pd


# Create an empty dictionary to store the attribute frequencies
attribute_counts = {}

# Iterate over each business
for attributes in business_df['attributes']:
    if attributes is not None:  # Check if attributes is not None
        # Iterate over each attribute
        for attribute in attributes.keys():
            # Increment the count for the attribute in the dictionary
            attribute_counts[attribute] = attribute_counts.get(attribute, 0) + 1

# Sort the attribute frequencies in descending order
sorted_attributes = sorted(attribute_counts.items(), key=lambda x: x[1], reverse=True)

# Print the three most popular attributes
print("Most popular attributes:")
for attribute, count in sorted_attributes[:7]:
    print(attribute, "-", count)



Most popular attributes:
BusinessAcceptsCreditCards - 119765
BusinessParking - 91085
RestaurantsPriceRange2 - 85314
BikeParking - 72638
RestaurantsTakeOut - 59857
WiFi - 56914
RestaurantsDelivery - 56282


In [56]:
import pandas as pd

# Choose three business attributes to split into columns
attributes_to_split = ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']

# Create an empty DataFrame to store the binary attribute columns
binary_attributes_df = pd.DataFrame()

# Iterate over each attribute
for attribute in attributes_to_split:
    # Create a column with binary values indicating whether the business has the attribute
    binary_attributes_df[attribute] = business_df['attributes'].apply(lambda x: int(attribute in x.keys()) if x is not None else 0)

# Add the business ID column to the resulting DataFrame
binary_attributes_df['business_id'] = business_df['business_id']
binary_attributes_df.head()

,BusinessAcceptsCreditCards,BikeParking,RestaurantsTakeOut,business_id
0,0,0,0,Pns2l4eNsfO8kk83dixA6A
1,1,0,0,mpf3x-BjTdTEA3yCZrAYPw
2,1,1,1,tUFrWirKiKi_TAnsVWINQQ
3,1,1,1,MTSW4McQd7CbVtyjqoe9mw
4,1,1,1,mWMc6_wTdE0EUBKIGXDVfA


In [57]:
sample_review = review_df.sample(n=30000)
sample_tip = tip_df.sample(n=30000)

In [58]:
import pandas as pd

# Concatenate the 'text' columns from review_data and tip_data
all_review_text = pd.concat([sample_review['text'], sample_tip['text']], axis=0)
all_tip_text = pd.concat([pd.Series(dtype='str'), sample_tip['text']], axis=0)

# Reset the index of all_review_text
all_review_text.reset_index(drop=True, inplace=True)

# Concatenate the 'business_id' columns from review_data and tip_data
all_business_ids = pd.concat([sample_review['business_id'], sample_tip['business_id']], axis=0)

# Reset the index of all_business_ids
all_business_ids.reset_index(drop=True, inplace=True)
all_tip_text.reset_index(drop=True, inplace=True)
all_review_text.reset_index(drop=True, inplace=True)

# Create a new DataFrame with separate columns for review and tip text data, and the business ID
features_df = pd.DataFrame({'business_id': all_business_ids, 'review_text': all_review_text, 'tip_text': all_tip_text})


In [59]:
#sampling
sample_attributes = binary_attributes_df.sample(n=60000)

In [60]:
# Merge features and labels based on business_id
data = pd.merge(features_df, sample_attributes, on='business_id')

In [61]:
from sklearn.model_selection import train_test_split

X = data[['review_text', 'tip_text']]  # Review and tip text as features
y = data[['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']]  # Labels (attributes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [62]:
# Replace missing values with empty strings
X_train['tip_text'].fillna('', inplace=True)
X_test['tip_text'].fillna('', inplace=True)
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_review = vectorizer.fit_transform(X_train['review_text'])
X_train_tip = vectorizer.transform(X_train['tip_text'])

# Transform the testing data
X_test_review = vectorizer.transform(X_test['review_text'])
X_test_tip = vectorizer.transform(X_test['tip_text'])



In [63]:
# Concatenate the review and tip features
X_train_features = hstack([X_train_review, X_train_tip])
X_test_features = hstack([X_test_review, X_test_tip])

In [64]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# Create a multiclassifier model
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))

# Fit the model on the training data
model.fit(X_train_features, y_train)

# Predict on the testing data
y_pred = model.predict(X_test_features)

In [65]:
# Calculate test accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)
# Print Classification Report
print(classification_report(y_test, y_pred))

Test Accuracy: 0.717948717948718
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      4510
           1       0.86      0.95      0.90      3893
           2       0.87      0.95      0.91      3594

   micro avg       0.89      0.96      0.93     11997
   macro avg       0.89      0.96      0.92     11997
weighted avg       0.89      0.96      0.93     11997
 samples avg       0.87      0.92      0.88     11997



In [66]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

# Initialize the random forest classifier model
model1 = RandomForestClassifier()

# Initialize the multi-output classifier
multi_output_model = MultiOutputClassifier(model1)

# Train the multi-output model
multi_output_model.fit(X_train_features, y_train)

# Predict the outputs for the training set
y_train_pred = multi_output_model.predict(X_train_features)

# Predict the outputs for the test set
y_test_pred = multi_output_model.predict(X_test_features)

# Evaluate the model
accuracy = multi_output_model.score(X_test_features, y_test)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_test_pred))


Accuracy: 0.71712158808933
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      4510
           1       0.83      0.99      0.90      3893
           2       0.82      0.98      0.89      3594

   micro avg       0.86      0.99      0.92     11997
   macro avg       0.86      0.99      0.92     11997
weighted avg       0.87      0.99      0.92     11997
 samples avg       0.86      0.95      0.88     11997



In [69]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Create a multiclassifier model
model2 = OneVsRestClassifier(SVC())

# Fit the model on the training data
model2.fit(X_train_features, y_train)

# Predict on the testing data
y_pred = model2.predict(X_test_features)

# Calculate test accuracy
accuracy1 = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy1)

# Print classification report
report = classification_report(y_test, y_pred)
print(report)


Test Accuracy: 0.7212572373862697
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      4510
           1       0.82      1.00      0.90      3893
           2       0.82      0.98      0.90      3594

   micro avg       0.86      0.99      0.92     11997
   macro avg       0.86      0.99      0.92     11997
weighted avg       0.86      0.99      0.92     11997
 samples avg       0.86      0.95      0.89     11997



In [70]:
class_names = ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']  # Replace with your actual class names

# Iterate over the predicted labels
for i, pred_label in enumerate(y_pred):
    # Get the predicted class indices for the current example
    pred_class_indices = np.where(pred_label == 1)[0]
    # Map the class indices to class names
    pred_class_names = [class_names[idx] for idx in pred_class_indices]
    
    # Print the predicted class label(s)
    print(f"Example {i+1}: Predicted Label(s): {pred_class_names}")


Example 1: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 2: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 3: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 4: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 5: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 6: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 7: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 8: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking']
Example 9: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 10: Predicted Label(s): ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut']
Example 11: Predicted Label(s):

In [71]:
from sklearn.metrics import confusion_matrix

# Generate the confusion matrix
cm = confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1))

# Print the confusion matrix
print(cm)


[[4716    0    0]
 [  80    0    0]
 [  40    0    0]]


-4616 true negative predictions for the 'BusinessAcceptsCreditCards' class, meaning that 4617 instances in the test set were correctly classified as not accepting credit cards for businesses.

-80 false negative predictions for the 'BusinessAcceptsCreditCards' class, meaning that 94 instances in the test set were incorrectly classified as not accepting credit cards for businesses.

-40  false negative predictions for the 'BusinessAcceptsCreditCards' class, meaning that 36 instances in the test set were incorrectly classified as not accepting credit cards for businesses.